In [1]:
import pandas as pd
import glob
import os
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('../summary_data/50bp/all_tools_TRB.csv')
data

,Sample,CDR3,nReads_TCR,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,total_reads_MIXCR,total_reads_IMREP,total_reads_TRUST4,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4,repertoire_type,class
0,sample01,CASSQVIRTPSRYGYTF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,0.0,32598.0,18795.0,0.000022,0.0,0.000000,0.0,monoclonal,T_cell_rich_monoclonal
1,sample01,CASSIRAGTSESYEQYF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,0.0,32598.0,18795.0,0.000022,0.0,0.000000,0.0,monoclonal,T_cell_rich_monoclonal
2,sample01,CASSNRGGAAYEQYF,3.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,0.0,32598.0,18795.0,0.000033,0.0,0.000000,0.0,monoclonal,T_cell_rich_monoclonal
3,sample01,CASSRGQSSEQFF,6.0,0.0,2.0,0.0,PBMC,T_cell_rich,90577.0,0.0,32598.0,18795.0,0.000066,0.0,0.000061,0.0,monoclonal,T_cell_rich_monoclonal
4,sample01,CASSLEGGYGYTF,2.0,0.0,0.0,0.0,PBMC,T_cell_rich,90577.0,0.0,32598.0,18795.0,0.000022,0.0,0.000000,0.0,monoclonal,T_cell_rich_monoclonal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134602,sample15,CASSYSGRAEETQYF,9.0,0.0,0.0,0.0,melanoma,T_cell_poor,749686.0,0.0,0.0,4.0,0.000012,0.0,0.000000,0.0,polyclonal,T_cell_poor_polyclonal
134603,sample15,CASSFGQLNTEAFF,34.0,0.0,0.0,0.0,melanoma,T_cell_poor,749686.0,0.0,0.0,4.0,0.000045,0.0,0.000000,0.0,polyclonal,T_cell_poor_polyclonal
134604,sample15,CASSLKGQGPQEQYF,59.0,0.0,0.0,0.0,melanoma,T_cell_poor,749686.0,0.0,0.0,4.0,0.000079,0.0,0.000000,0.0,polyclonal,T_cell_poor_polyclonal
134605,sample15,CASSPGGDFTYEQYF,78.0,0.0,0.0,0.0,melanoma,T_cell_poor,749686.0,0.0,0.0,4.0,0.000104,0.0,0.000000,0.0,polyclonal,T_cell_poor_polyclonal


In [3]:
def compute_frequency(samples_data, tool, discard_zero_freq_samples):

    samples = set(samples_data['Sample'])
    
    type_observed = tool
    
    data = samples_data.rename(columns={'frequency_TCR': 'frequency_truth'})
    data = data.rename(columns={'frequency_'+type_observed: 'frequency_observed'})
    data = data.rename(columns={'nReads_TCR': 'count_truth'})
    data = data.rename(columns={'nReads_'+type_observed: 'count_observed'})
    
    data = data.loc[data['frequency_truth'] > 0] #excluding clonotypes not present in ground truth

    file = open('../summary_data/50bp/capturing_per_class/capturing_'+type_observed+'.csv',"w")
        
    file.write("th,observed_portion_frequency\n")
    
    data_per_sample = {}

    for s in samples:
        
        data_per_sample[s] = data.loc[data['Sample'] == s, ['frequency_truth','frequency_observed','count_truth','count_observed']]

    for th in np.linspace(0., 1.0, 100000):

        s_portions = []

        for s in samples: 

            k_observed=0
            k_true=0

            s_observed=0
            s_true=0

            sample_data = data_per_sample[s]
            sample_data = sample_data.loc[sample_data['frequency_truth'] >= th]
            
            for index, cdr in sample_data.iterrows():

                freq = cdr['frequency_truth']
                
                if cdr['frequency_observed'] > 0: #check if observed sample has the clonotype
                        
                    k_observed += 1.0    #number of clonotypes with frequency greater or equal than th
                    s_observed += freq   #sum of frequencies of observed greater or equal than th
                        
                k_true += 1.0 #number of clonotypes with frequency greater or equal than th
                s_true += freq #sum of frequencies of TCR greater or equal than th

            if s_true > 0:
                s_portions.append(s_observed/s_true)
            elif discard_zero_freq_samples is False:
                s_portions.append(0.)

        file.write(str(th)+","+str(np.mean(s_portions))+"\n")

    file.close()
    print('done '+tool)

Discarding samples with no clonotypes from truth to compute the average portion

In [4]:
# T_cell_rich_monoclonal samples

T_cell_rich_monoclonal = data.loc[data['class'] == 'T_cell_rich_monoclonal']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(T_cell_rich_monoclonal, tool, True)

done MIXCR
done IMREP
done TRUST4


In [5]:
# T_cell_rich_polyclonal samples

T_cell_rich_polyclonal = data.loc[data['class'] == 'T_cell_rich_polyclonal']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(T_cell_rich_polyclonal, tool, True)

done MIXCR
done IMREP
done TRUST4


In [6]:
# T_cell_poor_polyclonal samples

T_cell_poor_polyclonal = data.loc[data['class'] == 'T_cell_poor_polyclonal']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(T_cell_poor_polyclonal, tool, True)

done MIXCR
done IMREP
done TRUST4
